In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.9 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [23]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-08-14 17:22:56,319] A new study created in memory with name: no-name-4f9a7425-bfbc-412e-8c48-c83a32d8526b
[I 2025-08-14 17:22:57,064] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 152, 'max_depth': 11}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-08-14 17:22:58,795] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 147, 'max_depth': 11}. Best is trial 1 with value: 0.7653631284916201.
[I 2025-08-14 17:22:59,986] Trial 2 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 99, 'max_depth': 10}. Best is trial 1 with value: 0.7653631284916201.
[I 2025-08-14 17:23:00,953] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 196, 'max_depth': 13}. Best is trial 3 with value: 0.7709497206703911.
[I 2025-08-14 17:23:01,908] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 193, 'max_depth': 19}. Best is trial 4 with value: 0.77467

In [24]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 128, 'max_depth': 16}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


**SAMPLERS IN OPTUNA**

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-08-14 17:17:29,090] A new study created in memory with name: no-name-885b7946-8bbe-4707-abc1-de8ab3a6e99a
[I 2025-08-14 17:17:29,750] Trial 0 finished with value: 0.7746741154562383 and parameters: {'n_estimators': 132, 'max_depth': 19}. Best is trial 0 with value: 0.7746741154562383.
[I 2025-08-14 17:17:30,591] Trial 1 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 197, 'max_depth': 3}. Best is trial 0 with value: 0.7746741154562383.
[I 2025-08-14 17:17:31,131] Trial 2 finished with value: 0.756052141527002 and parameters: {'n_estimators': 121, 'max_depth': 3}. Best is trial 0 with value: 0.7746741154562383.
[I 2025-08-14 17:17:31,877] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 153, 'max_depth': 14}. Best is trial 0 with value: 0.7746741154562383.
[I 2025-08-14 17:17:32,559] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 144, 'max_depth': 5}. Best is trial 0 with value: 0.77467411

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 119, 'max_depth': 7}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-08-14 17:18:43,177] A new study created in memory with name: no-name-b311dbb1-252d-4924-b43d-d5e2b657a975
[I 2025-08-14 17:18:43,736] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-14 17:18:44,852] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-14 17:18:45,254] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-14 17:18:45,932] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-14 17:18:46,428] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


**OPTUNA VISUALIZATIONS**

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [28]:
# 1. Optimization History
plot_optimization_history(study).show()

In [27]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [26]:
# 3. Slice Plot
plot_slice(study).show()

In [25]:
# 4. Contour Plot
plot_contour(study).show()

In [29]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models



In [30]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [31]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [32]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-08-14 17:29:05,522] A new study created in memory with name: no-name-8b7c7790-b59c-40c8-ab43-68b9b067c3d7
[I 2025-08-14 17:29:08,727] Trial 0 finished with value: 0.7486033519553073 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 249, 'learning_rate': 0.15403249996455212, 'max_depth': 14, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7486033519553073.
[I 2025-08-14 17:29:09,755] Trial 1 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 233, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 9, 'bootstrap': True}. Best is trial 1 with value: 0.7597765363128491.
[I 2025-08-14 17:29:10,874] Trial 2 finished with value: 0.7541899441340782 and parameters: {'classifier': 'RandomForest', 'n_estimators': 266, 'max_depth': 3, 'min_samples_split': 7, 'min_samples_leaf': 9, 'bootstrap': True}. Best is trial 1 with value: 0.7597765363128491.
[I 2025-08-14 17:29:11,689] Trial 3 

In [33]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.13034744809484988, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [34]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.748603,2025-08-14 17:29:05.525295,2025-08-14 17:29:08.727006,0 days 00:00:03.201711,NaN,NaN,GradientBoosting,NaN,NaN,0.154032,14.0,1.0,10.0,249.0,COMPLETE
1,1,0.759777,2025-08-14 17:29:08.727902,2025-08-14 17:29:09.755470,0 days 00:00:01.027568,NaN,True,RandomForest,NaN,NaN,NaN,6.0,9.0,7.0,233.0,COMPLETE
2,2,0.754190,2025-08-14 17:29:09.756282,2025-08-14 17:29:10.874019,0 days 00:00:01.117737,NaN,True,RandomForest,NaN,NaN,NaN,3.0,9.0,7.0,266.0,COMPLETE
3,3,0.752328,2025-08-14 17:29:10.876377,2025-08-14 17:29:11.689170,0 days 00:00:00.812793,NaN,True,RandomForest,NaN,NaN,NaN,8.0,2.0,6.0,175.0,COMPLETE
4,4,0.763501,2025-08-14 17:29:11.690029,2025-08-14 17:29:11.715345,0 days 00:00:00.025316,0.284012,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-08-14 17:29:33.285899,2025-08-14 17:29:33.313021,0 days 00:00:00.027122,0.152171,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.752328,2025-08-14 17:29:33.313564,2025-08-14 17:29:33.344847,0 days 00:00:00.031283,0.151558,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.789572,2025-08-14 17:29:33.345305,2025-08-14 17:29:33.369712,0 days 00:00:00.024407,0.134448,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.787709,2025-08-14 17:29:33.370296,2025-08-14 17:29:33.395757,0 days 00:00:00.025461,0.179746,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [35]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,80
GradientBoosting,10
RandomForest,10


In [36]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.745810
RandomForest,0.760708
SVM,0.777025


In [37]:
# 1. Optimization History
plot_optimization_history(study).show()

In [39]:
# 2. Slice Plot
plot_slice(study).show()

In [40]:
# 3. Hyperparameter Importance
plot_param_importances(study).show()